# 1 - Adjust base data

NetCDF data has already been combined and mapped to consistent depths. This Notebook will import data, truncate to desired depth range, check for data gaps and segment the dataset accordingly. Finally, data will be rotated according to site, small gaps interpolated, and filtered to acquire mean currents and residual flows. Output in NetCDF format for further processing.

## Imports

Import modules and data.

In [1]:
# import modules

import xarray as xr
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sig
import pandas as pd
for i in range(2):
    %matplotlib notebook

In [2]:
# import 'raw' data

adcp = 'Slope'     # Slope, Axis75, Axis55
season = 'Annual'  # Annual, Winter, Spring, Summer, Fall

if adcp == 'Slope':
    ds = xr.open_dataset(f'../Data/{adcp}All/{adcp}_all.nc')
    name = 'Slope'
elif adcp == 'Axis75':
    ds = xr.open_dataset(f'../Data/AxisAll/{adcp}/{adcp}_all.nc')
    name = 'Axis'   # for consistent naming between Axis 75 and 55 kHz
elif adcp == 'Axis55':
    ds = xr.open_dataset(f'../Data/AxisAll/{adcp}/{adcp}_all.nc')
    name = 'Axis'   # for consistent naming between Axis 75 and 55 kHz      
    
print(ds)

<xarray.Dataset>
Dimensions:  (depth: 48, time: 257962)
Coordinates:
  * depth    (depth) int64 3 11 19 27 35 43 51 ... 331 339 347 355 363 371 379
  * time     (time) datetime64[ns] 2009-09-24T19:52:30 ... 2019-05-16T22:07:30
Data variables:
    u        (time, depth) float64 ...
    v        (time, depth) float64 ...
    w        (time, depth) float64 ...
    temp     (time) float32 ...


In [3]:
# plot dataset to check data quality

# fig,ax = plt.subplots(figsize=(10,4))
# im = ax.pcolormesh(ds.time, -ds.depth, ds.u.T, rasterized=True, cmap='RdBu_r', vmin=-0.2, vmax=0.2)
# cbar = fig.colorbar(im, ax=ax, fraction=0.05, pad=0.01, aspect=40, extend='both')
# cbar.set_label('Velocity [m/s]')
# ax.set_xlabel('Time')
# ax.set_ylabel('Depth [m]')
# ax.set_title('Velocity data')
# plt.show()

## Depth

Truncate data to a specific depth interval to eliminate unreliable data, based on data quality checks regarding correlation and backscatter intensity.

In [4]:
# process to find nearby indices for desired depth values

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx      # returns index of nearest depth value

array = ds.depth         # input array to process 
if adcp=='Slope':
    upval = 0         # upper depth for data, metres (upper slope)
    lowval = 400        # lower depth for data (upper slope)
elif adcp=='Axis75':
    upval = 0         # upper depth for data, metres (axis)
    lowval = 1000        # lower depth for data (axis)
elif adcp=='Axis55':
    upval = 0            # upper depth for data, metres (axis)
    lowval = 1000        # lower depth for data (axis)
    
upidx = find_nearest(array, upval)                 # index of upper depth cutoff
lowidx = find_nearest(array,lowval)                # index of lower depth cutoff 

print("Index at upper depth cutoff:",upidx,"/ Value at upper depth cutoff:",-ds.depth.values[upidx],"metres")
print("Index at lower depth cutoff:",lowidx,"/ Value at lower depth cutoff:",-ds.depth.values[lowidx],"metres")

depth = np.array(ds.depth[upidx:lowidx+1])         # remove unwanted depths
dup_stamp = int(-depth[0])                         # depth stamps for use in output filenames
dlow_stamp = int(-depth[-1])

print("Length of new depth array: ",len(depth),'/ Upper limit:',dup_stamp,'metres','/ Lower limit:',dlow_stamp,'metres')   # new depth interval

Index at upper depth cutoff: 0 / Value at upper depth cutoff: -3 metres
Index at lower depth cutoff: 47 / Value at lower depth cutoff: -379 metres
Length of new depth array:  48 / Upper limit: -3 metres / Lower limit: -379 metres


## Time

Find specific time range and format dates. If significant *consecutive* NaN values are present, then split time series into segments for analysis processing.

In [10]:
# set date range

year = 2013

datestimes = pd.to_datetime(ds.time.values)                          # convert to datetime from datetime64
datestimes = pd.Series(datestimes)                                   # convert to pandas dataframe

start_date = dt.datetime(year,1,1)                                   # input start date in YYYY,MM,DD
end_date = dt.datetime(year+1,1,1)                                   # input end date in YYYY,MM,DD
start = datestimes[datestimes >= start_date].index[0]                # desired start date
end = datestimes[datestimes < end_date].index[-1]                    # desired end date
time_test = ds.time.values[start:end]                                # test desired interval
t_stamp = f'{datestimes.dt.year[start]}'                             # set year time stamp for output filenames
print("Desired time range:",np.min(time_test),np.max(time_test))     # print to check desired interval

Desired time range: 2013-01-01T00:07:30.000003328 2013-12-31T23:37:30.000000000
2013-12-31T23:37:30.000000000


In [11]:
# check time series for significant data gaps

depth_test = ds.depth[lowidx-5]                        # mid depth for NaN testing
print('Checked at depth',-depth_test.values,'m')

u_test = np.array(ds.u[start:end,lowidx-5])            # u data at this depth
counter = 0                                            # counter to keep track of # of consecutive NaN values
nan_list = [0]                                         # empty list to keep track of NaN indices

for i in range(len(time_test)):                        # loop to count consecutive NaN values
    if np.isnan(u_test[i])==True:                      # add to counter if NaN = true
        counter += 1
        if counter==100:                               # length of NaN gap
            dead = i-100                               # datetime series hits significant NaN interval
            nan_list.append(dead)                      # append end of good data index to nan_list
            print('Good data until:',np.max(time_test[dead+1]))
        elif counter > 100 and i < (len(time_test)-1):   # if NaNs, but before the end of the interval
            if np.isnan(u_test[i+1])==False:           # if next value is NOT a NaN
                nan_list.append(i+1)                   # append that index to nan_list as start of good data
                print('Good data resets at:',time_test[i])
    elif np.isnan(u_test[i])==False:                   # reset counter if NaN inconsistent
        counter = 0

if (len(nan_list)%2) != 0:
    nan_list.append(len(time_test))

u_test = np.array(ds.u[start:end,:])      # total u for interval and depth, to chop
v_test = np.array(ds.v[start:end,:])      # total v for interval and depth, to chop
u_seg = []                                             # empty list for u segment arrays
v_seg = []                                             # empty list for v segment arrays
t_seg = []                                             # empty list for u segment time intervals

for i in range(len(nan_list)):                         # for the number of NaN indices found
    if i%2 != 0:                                       # for every second index
        u_seg.append(np.array(u_test[nan_list[i-1]:nan_list[i],:]))      # create arrays of good u data
        v_seg.append(np.array(v_test[nan_list[i-1]:nan_list[i],:]))      # create arrays of good v data
        t_seg.append(np.array(time_test[nan_list[i-1]:nan_list[i]]))     # and their time intervals

n_seg = len(u_seg)
print('Number of segments:',n_seg)
for i in range(n_seg):
    print(f'Length of segment {i}:',len(t_seg[i]))

Checked at depth -339 m
Number of segments: 1
Length of segment 0: 35039


In [7]:
# plot specified time intervals to check data quality

# fig,ax = plt.subplots(figsize=(12,4))
# for i in range(n_seg):
#     im = ax.pcolormesh(t_seg[i], -depth, u_seg[i].T, rasterized=True, cmap='RdBu_r', vmin=-0.1, vmax=0.1)
# cbar = fig.colorbar(im, ax=ax, fraction=0.05, pad=0.01, aspect=40, extend='both')
# ax.set_xlim(start_date,end_date)
# cbar.set_label('Velocity [m/s]')
# ax.set_xlabel('Time')
# ax.set_ylabel('Depth [m]')
# ax.set_title('Velocity data')
# plt.show()

## Adjust
Data rotated based on a visual estimate of along-slope angle, as 30$^{\circ}$, for Upper Slope. Axis rotation based on lower canyon topography, as %%%. Data are then interpolated to deal with minor instances of NaN values, and filtered using a 40h low-pass Butterworth digital filter to extract the mean currents and residual (tidal) flows.

In [17]:
# process to rotate, interpolate, and filter raw data (no significant data gaps)

uorig,ulp,vorig,vlp = [],[],[],[]   # empty lists for adjusted data

for i in range(n_seg):

    # rotate Upper Slope data (y is already fairly along-canyon for Axis)
    
    if adcp == 'Slope':
        theta_along_slope = np.radians(-30)         # rotation angle in radians, -30 degrees to rotate AXES not velocities
        u_old = u_seg[i] 
        v_old = v_seg[i]                                         
        u_rot = u_old*np.cos(theta_along_slope) - v_old*np.sin(theta_along_slope)               # u_new = Re(rotated vector)
        v_rot = u_old*np.sin(theta_along_slope) + v_old*np.cos(theta_along_slope)               # v_new = Im(rotated vector)     
    elif adcp == 'Axis75' or adcp == 'Axis55':
        u_rot = u_seg[i]
        v_rot = v_seg[i]

    # filter small NaN instances from data

    t = len(t_seg[i])                        # number of time data points after checking for consistent NaN intervals
    d = len(depth)                           # number of depth data points after removing unwanted depths

    uorig_temp = np.empty([t,d])             # empty array for interpolated & rotated u data
    vorig_temp = np.empty([t,d])             # empty array for interpolated & rotated v data

    for j in range(d):                       # loop to interpolate small gaps at each depth
        utemp = pd.Series(u_rot[:,j])
        uint = utemp.interpolate(method="linear", limit = 100, limit_direction='both')
        uorig_temp[:,j] = uint               # set interpolated data to original array
        vtemp = pd.Series(v_rot[:,j])
        vint = vtemp.interpolate(method="linear", limit = 100, limit_direction='both')
        vorig_temp[:,j] = vint               # set interpolated data to original array

    # low pass Butterworth filter for 40 hour cut-off to remove 30 hour tides

    fs = 1.111e-3                            # samples per SECOND for entire time series
    fc = 6.944e-6                            # 40 hour low pass filter cut-off
    Wn = fc                                  # cut-off frequencies
    sos = sig.butter(8, Wn,'lowpass', fs=fs, output='sos')   # digital butterworth filter
    #w, h = sig.sosfreqz(sos)                   # to plot filter response

    ulp_temp = np.empty([t,d])               # empty array for low-pass filtered u values
    vlp_temp = np.empty([t,d])               # empty array for low-pass filtered v values

    for j in range(d):                       # loop for filtered and residual velocities
        ufilt = np.copy(uorig_temp[:,j])     # copy unfiltered array
        ulp_temp[:,j] = sig.sosfiltfilt(sos, ufilt)  # apply low pass filter

        vfilt = np.copy(vorig_temp[:,j])           # repeat for v
        vlp_temp[:,j] = sig.sosfiltfilt(sos, vfilt)
        
    uorig.append(uorig_temp)                  # append cleaned data to lists
    vorig.append(vorig_temp)
    ulp.append(ulp_temp)
    vlp.append(vlp_temp)

In [19]:
# plot filter response

#plt.semilogx(w*fs/(2*np.pi),abs(h))

In [11]:
# plot specified time intervals to check data quality

# fig,ax = plt.subplots(figsize=(12,4))
# for i in range(n_seg):
#     im = ax.pcolormesh(t_seg[i], -depth, uorig[i].T, rasterized=True, cmap='RdBu_r', vmin=-0.1, vmax=0.1)
# cbar = fig.colorbar(im, ax=ax, fraction=0.05, pad=0.01, aspect=40, extend='both')
# ax.set_xlim(start_date,end_date)
# cbar.set_label('Velocity [m/s]')
# ax.set_xlabel('Time')
# ax.set_ylabel('Depth [m]')
# ax.set_title('Velocity data')
# plt.show()

## Save
Save key values and arrays to NetCDF format using xarray.

In [11]:
# save to .nc files

for i in range(n_seg):
    ds_out = xr.Dataset( 
        data_vars=dict(
            uorig=(['t_seg','depth'], uorig[i]),    # adjusted data
            vorig=(['t_seg','depth'], vorig[i]),
            ulp=(['t_seg','depth'], ulp[i]),        # low-pass data
            vlp=(['t_seg','depth'], vlp[i]),
        ),
        coords=dict(
            depth=depth,                 # depth values
            t_seg=t_seg[i],              # datetime values in segments
        ),
        attrs=dict(
            description=f'Adjusted data for {adcp} {t_stamp} segment {i} (from 0 to {n_seg - 1}).',
            adcp=adcp,                   # adcp
            upidx=upidx,                 # upper depth index
            lowidx=lowidx,               # lower depth index
            dup_stamp=dup_stamp,         # upper depth stamp
            dlow_stamp=dlow_stamp,       # lower depth stamp
            t_stamp=t_stamp,             # year stamp
            start_date=f'{start_date}',  # start date
            end_date=f'{end_date}',      # end date
            t=t,                         # length of time series
            d=d,                         # length of depth series
            n_seg=n_seg,                 # total number of segments
        ),
    ) 
    ds_out.to_netcdf(f'../Data/data/adj/adj_{adcp}_{t_stamp}_{i}.nc')